<a href="https://colab.research.google.com/github/jacobgreen4477/The-4th-ETRI-AI-Human-Understanding-Competition/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%83%90%EC%83%89%EA%B2%B0%EA%B3%BC%EB%B0%98%EC%98%81_0_714120513489859.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In our study, we used smartphones, smartwatches, sleep sensors, and self-recording apps to collect daily life logs and sleep health records of study participants in 2024.The data collection procedures and methods followed a similar approach to those used in previous studies. Here, we pu﻿blicly provide the following 12 data items, which comprise a total of 700 days' worth of lifelog data, strictly for non-commercial and academic research purposes only.
- mACStatus: Indicates whether the smartphone is currently being charged.
- mActivity: Value calculated by the Google Activity Recognition API.
- mAmbience: Ambient sound identification labels and their respective probabilities.
- mBle: Bluetooth devices around individual subject.
- mGps: Multiple GPS coordinates measured within a single minute using the smartphone.
- mLight: Ambient light measured by the smartphone.
- mScreenStatus: Indicates whether the smartphone screen is in use.
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mWifi: Wifi devices around individual subject.
- wHr: Heart rate readings recorded by the smartwatch.
- wLight: Ambient light measured by the smartwatch.
- wPedo: Step data recorded by the smartwatch.

For the purpose of training a learning model to predict sleep health, fatigue, and stress, the following six metrics were derived from sleep sensor data and self-reported survey records. Each metric consists of values categorized into either two levels (0, 1) or three levels (0, 1, 2), depending on the specific metric. The detailed classification criteria for each metric's levels will be provided in a separate document.These
metrics assign a value of 0 for sleep records that do not meet the recommended guidelines.For instance, the first questionnaire metric (Q1) is assigned a value of 1 on days when an
individual’s self-reported sleep quality exceeds their average over the experimental period, and 0 when it
falls below that average. Similarly, the second and third metrics (Q2 and Q3) are assigned a value of 0
on days when the participant’s fatigue and stress levels, respectively, exceed their average, and a value of
1 when these levels are below average.

- Q1: Overall sleep quality as perceived by a subject immediately after waking up.
- Q2: Physical fatigue of a subject just before sleep.
- Q3: Stress level experienced by a subject just before sleep.
- S1: Adherence to sleep guidelines for total sleep time (TST).
- S2: Adherence to sleep guidelines for sleep efficiency (SE).
- S3: Adherence to sleep guidelines for sleep onset latency (SOL, or SL).

수면 건강, 피로, 스트레스 예측을 위한 학습 모델을 훈련시키기 위해, 수면 센서 데이터와 자기 보고식 설문 기록을 기반으로 다음의 6가지 지표를 도출했습니다.
각 지표는 해당 항목에 따라 두 수준(0, 1) 또는 세 수준(0, 1, 2)으로 구분된 값을 가집니다.
각 지표의 세부 분류 기준은 별도의 문서에서 제공될 예정입니다.

- Q1: 기상 직후 본인이 인지한 전반적인 수면의 질
 - 0: 개인 평균 이하
 - 1: 개인 평균 이상
- Q2: 취침 직전 본인이 느낀 신체적 피로 수준
 - 0: 높은 피로 수준
 - 1: 낮은 피로 수준
- Q3: 취침 직전 본인이 느낀 스트레스 수준
 - 0: 높은 스트레스 수준
 - 1: 낮은 스트레스 수준
- S1: 총 수면 시간(TST) 가이드라인을 준수했는지 3LEVELS
 - 0: 가이드라인 미준수
 - 1: 가이드라인 부분적 준수
 - 2: 가이드라인 완전 준수
- S2: 수면 효율(SE) 가이드라인을 준수했는지 여부
- (SE: 잠자리에 누워 있었던 전체 시간 대비, 실제로 잠든 시간의 비율)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수
- S3: 수면 잠들기 지연 시간(SOL 또는 SL) 가이드라인을 준수했는지 여부
- (SOL: 잠자리에 누운 순간부터 실제로 잠드는 데까지 걸린 시간)
 - 0: 가이드라인 미준수
 - 1: 가이드라인 준수

### 📦 라이브러리

In [1]:
! pip install haversine
! pip install optuna
! pip install category_encoders
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
from collections import Counter
from scipy.stats import entropy
from haversine import haversine  # 설치 필요: pip install haversine

warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.3 MB/s eta 0:00:00


In [2]:
import re
import ast
from tqdm import tqdm  # ← 추가
from math import radians, cos, sin, asin, sqrt
from datetime import time
from datetime import timedelta
from functools import reduce
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import glob
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from category_encoders import TargetEncoder

# seed 고정
SD = 42
random.seed(SD)
np.random.seed(SD)
os.environ['PYTHONHASHSEED'] = str(SD)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

### 📦 데이터 읽기

In [6]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

### ✔️ mACStatus 핸드폰 충전상태
- Indicates whether the smartphone is currently being charged.
- m_charging : 0/1 상태
- 핸드폰이 오랫 동안 충전했다는 의미?
 - 한 자리에 장시간 머물러 있었다.
 - 핸드폰을 장시간 사용하지 않았다.  

In [ ]:
mACStatus['lifelog_date'] = mACStatus['timestamp'].astype(str).str[:10]
mACStatus.head(1)

,subject_id,timestamp,m_charging,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
def process_mACStatus(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])

    results = []

    for (subj, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        status = group['m_charging'].values  # 0/1 상태
        times = group['timestamp'].values

        ratio_charging = status.mean()
        sum_charging = status.sum()

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum()

        # 연속된 1 상태 길이들
        lengths = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                lengths.append(current_len)
                current_len = 0
        if current_len > 0:
            lengths.append(current_len)

        avg_charging_duration = np.mean(lengths) if lengths else 0
        max_charging_duration = np.max(lengths) if lengths else 0

        results.append({
            'subject_id': subj,
            'lifelog_date': lifelog_date,
            'charging_ratio': ratio_charging,
            'charging_sum': sum_charging,
            'charging_transitions': transitions,
            'avg_charging_duration': avg_charging_duration,
            'max_charging_duration': max_charging_duration,
        })

    return pd.DataFrame(results)

mACStatus2 = process_mACStatus(mACStatus)

# check
print(f'# mACStatus2 shape: {mACStatus2.shape}')
mACStatus2.head(1)

# mACStatus2 shape: (700, 7)


,subject_id,lifelog_date,charging_ratio,charging_sum,charging_transitions,avg_charging_duration,max_charging_duration
0,id01,2024-06-26,0.2159,147,22,13.3636,41


### ✔️ mActivity 추정행동
- Value calculated by the Google Activity Recognition API.
 - 0 : IN_VEHICLE
 - 1 : ON_BICYCLE
 - 2 : ON_FOOT
 - 3 : STILL (not moving)
 - 4 : UNKNOWN
 - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
 - 7 : WALKING
 - 8 : RUNNING
- 근무시간   : 오전 7시부터 오후 6시까지
- 근무외시간 : 오후6시부터 12시까지

In [ ]:
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]
mActivity.head()

,subject_id,timestamp,m_activity,lifelog_date
0,id01,2024-06-26 12:03:00,4,2024-06-26
1,id01,2024-06-26 12:04:00,0,2024-06-26
2,id01,2024-06-26 12:05:00,0,2024-06-26
3,id01,2024-06-26 12:06:00,0,2024-06-26
4,id01,2024-06-26 12:07:00,0,2024-06-26


In [ ]:
# def process_mActivity_by_timezones(df):
#     df = df.copy()
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     df['lifelog_date'] = df['timestamp'].dt.date
#     df['hour'] = df['timestamp'].dt.hour

#     def summarize(group_df, prefix=''):
#         summary = []
#         for (subj, date), group in group_df.groupby(['subject_id', 'lifelog_date']):
#             counts = group['m_activity'].value_counts(normalize=True)
#             counts2 = group['m_activity'].value_counts()
#             row = {'subject_id': subj, 'lifelog_date': date}

#             for i in range(9):
#                 row[f'{prefix}activity_{i}_ratio'] = counts.get(i, 0)
#                 row[f'{prefix}activity_{i}_count'] = counts2.get(i, 0)

#             row[f'{prefix}dominant_activity'] = group['m_activity'].mode()[0] if not group['m_activity'].mode().empty else None
#             row[f'{prefix}num_unique_activities'] = group['m_activity'].nunique()

#             summary.append(row)
#         return pd.DataFrame(summary)

#     # 전체 데이터 요약 (prefix 없음)
#     total_summary = summarize(df)

#     # 취침시간 (01:00 ~ 05:00)
#     sleeptime_df = df[(df['hour'] >= 1) & (df['hour'] <= 5)]
#     sleeptime_summary = summarize(sleeptime_df, prefix='sleeptime_')

#     # subject_id와 lifelog_date 기준으로 모두 합치기
#     result = total_summary.copy()
#     result = pd.merge(result, sleeptime_summary, on=['subject_id', 'lifelog_date'], how='left')

#     # 컬럼 정렬
#     cols = ['subject_id', 'lifelog_date'] + [col for col in result.columns if col not in ['subject_id', 'lifelog_date']]
#     result = result[cols]

#     # 결측 처리
#     result = result.fillna(0)

#     return result

# mActivity2 = process_mActivity_by_timezones(mActivity)

# # check
# print(f'# mActivity2 shape: {mActivity2.shape}')
# mActivity2.head(1)

In [ ]:
def process_mActivity(df):
    """
    # 포함
    - 0 : IN_VEHICLE
    - 1 : ON_BICYCLE
    - 2 : ON_FOOT
    - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
    - 7 : WALKING
    - 8 : RUNNING

    # 제외
    - 3 : STILL (not moving)
    - 4 : UNKNOWN
    """
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        row = {'subject_id': subj, 'lifelog_date': date}

        # 전체 시간에서 1,2,7,8
        a1 = group[group['m_activity'].isin([1,2,7,8])]
        row['all_WALKING_n_ETC_minutes'] = len(a1)

        # 전체 시간에서 0 (IN_VEHICLE)
        a2 = group[group['m_activity'].isin([0])]
        row['all_VEHICLE_minutes'] = len(a2)

        # 전체 시간에서 유효한 활동
        all_valid = group[group['m_activity'].isin([0, 1, 2, 5, 7, 8])]
        row['all_ACTIVITY_minutes'] = len(all_valid)

        # sleeptime 0~5시 에서 유효한 활동
        dawn_valid = all_valid[(all_valid['hour'] >= 0) & (all_valid['hour'] <5)]
        row['dawn_ACTIVITY_minutes'] = len(dawn_valid)

        results.append(row)

    return pd.DataFrame(results)

In [ ]:
mActivity2 = process_mActivity(mActivity)

# check
print(f'# mActivity2 shape: {mActivity2.shape}')
mActivity2.head(1)

# mActivity2 shape: (700, 6)


,subject_id,lifelog_date,all_WALKING_n_ETC_minutes,all_VEHICLE_minutes,all_ACTIVITY_minutes,dawn_ACTIVITY_minutes
0,id01,2024-06-26,32,89,121,0


### ✔️ mAmbience 추정주변소리
- Ambient sound identification labels and their respective probabilities.
- 무슨 소리가 난게 중요할까?
- 새벽에 무슨 소리던지 소리가 난게 중요한 걸까?
- 여러 가지 소리 중에 노이즈도 포함되어 있을까?

In [ ]:
def extract_labels_and_probs(row):
    items = row['m_ambience']
    labels = [item[0] for item in items]
    probs = [item[1] for item in items]
    return pd.Series({'labels': labels, 'prob': probs})

mAmbience[['labels', 'prob']]  = mAmbience.apply(extract_labels_and_probs, axis=1)
mAmbience['lifelog_date'] = mAmbience['timestamp'].astype(str).str[:10]
mAmbience = mAmbience.drop(columns=['m_ambience'])
mAmbience.head(1)

,subject_id,timestamp,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26


In [ ]:
# def process_mAmbience(df, top_n=3, special_labels=None):
#     df = df.copy()

#     # 시간 파생변수
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     df['hour'] = df['timestamp'].dt.hour
#     df['weekday'] = df['timestamp'].dt.weekday
#     df['is_weekend'] = df['weekday'] >= 5

#     def map_time_period(row):
#         if 0 <= row['hour'] < 7:
#             return 'sleeptime'
#         elif 7 <= row['hour'] < 18:
#             return 'worktime'
#         else:
#             return 'afterwork'

#     df['time_period'] = df.apply(map_time_period, axis=1)

#     # 타임스탬프 차이(초단위)
#     df = df.sort_values(['subject_id', 'timestamp'])
#     df['duration_sec'] = df.groupby(['subject_id'])['timestamp'].diff().dt.total_seconds()
#     df['duration_sec'] = df['duration_sec'].fillna(0)

#     def process_group(group):
#         result = {}

#         time_period = group['time_period'].iloc[0]  # 그룹별 time_period 가져오기

#         labels = group['labels'].tolist()
#         probs = group['prob'].tolist()
#         durations = group['duration_sec'].tolist()

#         flat_labels = [label for labels_list in labels for label in labels_list]
#         flat_probs = [float(prob) for probs_list in probs for prob in probs_list]
#         repeated_durations = [dur for labels_list, dur in zip(labels, durations) for _ in labels_list]

#         if not flat_labels or not flat_probs:
#             return pd.Series()

#         flat_probs = np.array(flat_probs)

#         # prefix 설정
#         prefix = f'{time_period}_'

#         result[prefix + 'max_label'] = flat_labels[np.argmax(flat_probs)]
#         result[prefix + 'max_prob'] = np.max(flat_probs)
#         result[prefix + 'entropy'] = entropy(flat_probs, base=2)
#         result[prefix + 'label_count'] = len(flat_labels)

#         # 정규식
#         vehicle_pattern = re.compile(r'car|vehicle|truck|bus|motorcycle|bicycle|boat|ship|train|subway|aircraft|helicopter|engine', re.IGNORECASE)
#         environment_pattern = re.compile(r'outside|inside|environment|ocean|rain|waterfall|wind|fire|waves|rustling|earthquake|explosion|thunder|smoke', re.IGNORECASE)
#         human_pattern = re.compile(r'speech|baby|child|laughter|crying|shout|screaming|groan|cough|hubbub|snoring|whimper|talking|singing|giggle|chatter|babbling', re.IGNORECASE)
#         music_pattern = re.compile(r'music|song|singing|choir|instrument|guitar|piano|violin|vocal|jazz|hip hop|pop|rock|opera|blues|folk|electronic|dubstep|reggae|house|metal|disco|dance|flamenco|trance|techno|saxophone|synthesizer|harp|accordion|clarinet|flute|drum|tabla|tambourine|maraca|steelpan|xylophone|orchestra|band', re.IGNORECASE)

#         # 분류
#         vehicle_related = [(label, dur) for label, dur in zip(flat_labels, repeated_durations) if vehicle_pattern.search(label)]
#         environment_related = [(label, dur) for label, dur in zip(flat_labels, repeated_durations) if environment_pattern.search(label)]
#         human_related = [(label, dur) for label, dur in zip(flat_labels, repeated_durations) if human_pattern.search(label)]
#         music_related = [(label, dur) for label, dur in zip(flat_labels, repeated_durations) if music_pattern.search(label)]

#         # has (존재 여부)
#         result[prefix + 'has_vehicle_related'] = int(len(vehicle_related) > 0)
#         result[prefix + 'has_environment_related'] = int(len(environment_related) > 0)
#         result[prefix + 'has_human_related'] = int(len(human_related) > 0)
#         result[prefix + 'has_music_related'] = int(len(music_related) > 0)

#         # sum (개수)
#         result[prefix + 'sum_vehicle_related'] = len(vehicle_related)
#         result[prefix + 'sum_environment_related'] = len(environment_related)
#         result[prefix + 'sum_human_related'] = len(human_related)

#         return pd.Series(result)

#     features = df.groupby(['subject_id', 'lifelog_date', 'time_period']).apply(process_group).reset_index()
#     features = features.pivot(index=['subject_id', 'lifelog_date'], columns='level_3', values=0).reset_index()

#     return features

In [ ]:
def process_mAmbience(df):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    df['time_period'] = df['hour'].apply(lambda h: 'sleeptime' if 0 <= h < 5 else 'activehour')

    # explode labels
    exploded = df.explode('labels')

    # unique label count
    unique_labels = (
        exploded.groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .nunique()
        .reset_index(name='unique_label_count')
    )

    # snor 포함 라벨 count
    snor_labels = (
        exploded[exploded['labels'].astype(str).str.contains('snor', case=False, na=False)]
        .groupby(['subject_id', 'lifelog_date', 'time_period'])['labels']
        .count()
        .reset_index(name='snor_count')
    )

    # 병합
    result = pd.merge(unique_labels, snor_labels, on=['subject_id', 'lifelog_date', 'time_period'], how='outer').fillna(0)
    result['snor_count'] = result['snor_count'].astype(int)
    result = result.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    result.columns = [f"{tp}_{metric}" for metric, tp in result.columns]
    result = result.reset_index()

    return result

In [ ]:
mAmbience2 = process_mAmbience(mAmbience)

# check
print(f'# mAmbience2 shape: {mAmbience2.shape}')
mAmbience2.head(1)

# mAmbience2 shape: (700, 6)


,subject_id,lifelog_date,activehour_unique_label_count,sleeptime_unique_label_count,activehour_snor_count,sleeptime_snor_count
0,id01,2024-06-26,265.0000,NaN,4.0000,NaN


### ✔️ mBle 블루투스
- Bluetooth devices around individual subject.
 - 7936 : Wearable, Headset, AV Device
 - 1796 : Peripheral (입력장치) 계열
 - 0 : 정보 없음 또는 알 수 없음(Unknown)
 - 1084 : Audio/Video (스피커, 헤드셋, 이어폰, TV 등)
 - 524 : Phone (휴대폰, 스마트폰)
 - 1060 : Headphones
 - 284 : commputer (PC, 노트북, PDA)

In [ ]:
def extract_mble_info(row):
    m_data = row['m_ble']
    address = [item['address'] for item in m_data]
    device_class = [item['device_class'] for item in m_data]
    rssi = [item['rssi'] for item in m_data]
    return pd.Series({'address': address, 'device_class': device_class, 'rssi': rssi})

mBle[['address','device_class','rssi']] = mBle.apply(extract_mble_info, axis=1)
mBle['lifelog_date'] = mBle['timestamp'].astype(str).str[:10]
mBle.head(1)

,subject_id,timestamp,m_ble,address,device_class,rssi,lifelog_date
0,id01,2024-06-26 12:13:00,"[{'address': '00:15:7C:11:80:8D', 'device_class': '0', 'rssi': -82}, {'address': '01:B1:D2:20:9E:3A', 'device_class': '0', 'rssi': -61}, {'address': '04:33:1F:D9:C1:50', 'device_class': '0', 'rssi': -86}, {'address': '06:5C:2D:BC:39:BE', 'device_class': '0', 'rssi': -75}, {'address': '09:42:21:0D:AD:DF', 'device_class': '0', 'rssi': -70}, {'address': '0B:66:0D:D5:9C:4A', 'device_class': '0', 'rssi': -89}, {'address': '10:B5:88:E7:85:69', 'device_class': '0', 'rssi': -89}, {'address': '13:F0:CA:3B:DB:EF', 'device_class': '0', 'rssi': -77}, {'address': '1A:23:C0:8F:43:4D', 'device_class': '0', 'rssi': -66}, {'address': '24:11:53:BB:62:89', 'device_class': '1796', 'rssi': -37}, {'address': '24:2D:F0:EE:1E:D0', 'device_class': '0', 'rssi': -85}, {'address': '26:0C:48:28:15:77', 'device_class': '0', 'rssi': -63}, {'address': '27:C1:C0:8B:82:C9', 'device_class': '0', 'rssi': -88}, {'address': '28:9C:11:73:39:05', 'device_class': '0', 'rssi': -30}, {'address': '34:40:DE:35:F8:65', 'device_class': '0', 'rssi': -93}, {'address': '35:0A:59:BF:75:F5', 'device_class': '0', 'rssi': -72}, {'address': '41:A6:C4:20:E3:2C', 'device_class': '7936', 'rssi': -83}, {'address': '42:6B:51:95:1B:D4', 'device_class': '0', 'rssi': -77}, {'address': '44:B2:0B:78:04:0F', 'device_class': '0', 'rssi': -69}, {'address': '45:37:48:E2:7F:CC', 'device_class': '0', 'rssi': -87}, {'address': '4E:1B:C2:DF:C5:87', 'device_class': '0', 'rssi': -76}, {'address': '4E:9F:1B:A9:56:5D', 'device_class': '0', 'rssi': -66}, {'address': '50:63:B0:82:07:00', 'device_class': '0', 'rssi': -86}, {'address': '53:13:6C:4F:04:D2', 'device_class': '0', 'rssi': -69}, {'address': '54:15:89:95:27:44', 'device_class': '7936', 'rssi': -71}, {'address': '56:0E:2E:B0:D4:11', 'device_class': '0', 'rssi': -61}, {'address': '5A:7A:2E:42:03:B1', 'device_class': '0', 'rssi': -82}, {'address': '5A:9D:3E:AB:38:C6', 'device_class': '0', 'rssi': -83}, {'address': '5E:A6:8E:B8:74:74', 'device_class': '0', 'rssi': -84}, {'address': '5F:BC:08:0F:C1:6A', 'device_class': '0', 'rssi': -87}, {'address': '62:E1:9D:41:F4:AE', 'device_class': '0', 'rssi': -73}, {'address': '67:23:FE:88:69:A8', 'device_class': '0', 'rssi': -88}, {'address': '68:EC:C5:0C:D1:C1', 'device_class': '0', 'rssi': -78}, {'address': '6B:28:DA:C0:1B:29', 'device_class': '0', 'rssi': -75}, {'address': '6F:0B:91:00:33:19', 'device_class': '0', 'rssi': -80}, {'address': '70:7A:4B:82:44:90', 'device_class': '0', 'rssi': -88}, {'address': '7B:62:D4:5B:59:D3', 'device_class': '0', 'rssi': -74}, {'address': '7B:BE:A4:9D:FD:11', 'device_class': '0', 'rssi': -72}, {'address': '7F:FD:C4:00:77:7D', 'device_class': '0', 'rssi': -52}, {'address': 'C4:F0:92:C8:F1:8D', 'device_class': '7936', 'rssi': -87}, {'address': 'C7:3F:2C:7B:86:66', 'device_class': '7936', 'rssi': -89}]","[00:15:7C:11:80:8D, 01:B1:D2:20:9E:3A, 04:33:1F:D9:C1:50, 06:5C:2D:BC:39:BE, 09:42:21:0D:AD:DF, 0B:66:0D:D5:9C:4A, 10:B5:88:E7:85:69, 13:F0:CA:3B:DB:EF, 1A:23:C0:8F:43:4D, 24:11:53:BB:62:89, 24:2D:F0:EE:1E:D0, 26:0C:48:28:15:77, 27:C1:C0:8B:82:C9, 28:9C:11:73:39:05, 34:40:DE:35:F8:65, 35:0A:59:BF:75:F5, 41:A6:C4:20:E3:2C, 42:6B:51:95:1B:D4, 44:B2:0B:78:04:0F, 45:37:48:E2:7F:CC, 4E:1B:C2:DF:C5:87, 4E:9F:1B:A9:56:5D, 50:63:B0:82:07:00, 53:13:6C:4F:04:D2, 54:15:89:95:27:44, 56:0E:2E:B0:D4:11, 5A:7A:2E:42:03:B1, 5A:9D:3E:AB:38:C6, 5E:A6:8E:B8:74:74, 5F:BC:08:0F:C1:6A, 62:E1:9D:41:F4:AE, 67:23:FE:88:69:A8, 68:EC:C5:0C:D1:C1, 6B:28:DA:C0:1B:29, 6F:0B:91:00:33:19, 70:7A:4B:82:44:90, 7B:62:D4:5B:59:D3, 7B:BE:A4:9D:FD:11, 7F:FD:C4:00:77:7D, C4:F0:92:C8:F1:8D, C7:3F:2C:7B:86:66]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1796, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7936, 7936]","[-82, -61, -86, -75, -70, -89, -89, -77, -66, -37, -85, -63, -88, -30, -93, -72, -83, -77, -69, -87, -76, -66, -86, -69, -71, -61, -82, -83, -84, -87, -73, -88, -78, -75, -80, 

In [ ]:
def process_mBle(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 0 <= row['hour'] < 7:
            return 'sleeptime'
        elif 7 <= row['hour'] < 18:
            return 'worktime'
        else:
            return 'afterwork'

    df['time_period'] = df.apply(map_time_period, axis=1)

    features = []

    for idx, row in df.iterrows():
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble']

        rssi_list = []
        class_0_cnt = 0
        class_other_cnt = 0

        for device in entry:
            try:
                rssi = int(device['rssi'])
                rssi_list.append(rssi)

                device_class = str(device['device_class'])
                if device_class == '0':
                    class_0_cnt += 1
                else:
                    class_other_cnt += 1
            except:
                continue  # malformed record

        feature = {
            'subject_id': row['subject_id'],
            'lifelog_date': row['lifelog_date'],
            'time_period': row['time_period'],
            'ble_class_unknwn_cnt': class_0_cnt,
            'ble_class_others_cnt': class_other_cnt,
            'ble_count': len(rssi_list),
            'ble_rssi_mean': np.mean(rssi_list) if rssi_list else np.nan,
            'ble_rssi_min': np.min(rssi_list) if rssi_list else np.nan,
            'ble_rssi_max': np.max(rssi_list) if rssi_list else np.nan,
        }
        features.append(feature)

    return pd.DataFrame(features)

def summarize_mBle_daily(df):

    # row 단위 BLE feature 추출
    df = process_mBle(df)

    # 하루 + 시간대별로 groupby
    grouped = df.groupby(['subject_id', 'lifelog_date', 'time_period']).agg({
        'ble_class_unknwn_cnt': 'sum',
        'ble_class_others_cnt': 'sum',
        'ble_rssi_mean': 'mean',
        'ble_rssi_min': 'min',
        'ble_rssi_max': 'max',
    }).reset_index()

    # 총합 구해서 비율 계산
    total_cnt = grouped['ble_class_unknwn_cnt'] + grouped['ble_class_others_cnt']
    grouped['ble_class_unknwn_ratio'] = grouped['ble_class_unknwn_cnt'] / total_cnt.replace(0, np.nan)
    grouped['ble_class_others_ratio'] = grouped['ble_class_others_cnt'] / total_cnt.replace(0, np.nan)

    # 필요 없는 cnt 컬럼 제거
    grouped.drop(columns=[
        'ble_class_unknwn_cnt',
        'ble_class_others_cnt'
    ], inplace=True)

    # pivot해서 time_period별로 펼치기
    final = grouped.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    final.columns = ['_'.join(col).strip() for col in final.columns.values]
    final = final.reset_index()

    return final

In [ ]:
mBle2 = summarize_mBle_daily(mBle)

# check
print(f'\n # mBle2 shape: {mBle2.shape}')
mBle2.head(1)


 # mBle2 shape: (651, 17)


,subject_id,lifelog_date,ble_rssi_mean_afterwork,ble_rssi_mean_sleeptime,ble_rssi_mean_worktime,ble_rssi_min_afterwork,ble_rssi_min_sleeptime,ble_rssi_min_worktime,ble_rssi_max_afterwork,ble_rssi_max_sleeptime,ble_rssi_max_worktime,ble_class_unknwn_ratio_afterwork,ble_class_unknwn_ratio_sleeptime,ble_class_unknwn_ratio_worktime,ble_class_others_ratio_afterwork,ble_class_others_ratio_sleeptime,ble_class_others_ratio_worktime
0,id01,2024-06-26,-76.2155,NaN,-75.0522,-92.0000,NaN,-94.0000,-43.0000,NaN,-27.0000,0.9237,NaN,0.9421,0.0763,NaN,0.0579


### ✔️ mGps, GPS 기반 핸드폰 위치
- Multiple GPS coordinates measured within a single minute using the smartphone.
- speed가 1보다 큰경우 정지 상태가 아니고 움직이고 있다고 판단
 - 0.5-2 : 걸어서 이동하는 경우  
 - 2-5 : 조깅
 - 5 이상 : 차를 타고 이동하는 경우

- speed가 0.5-2사이를 하루에 몇분동안 지속했는지?
- speed가 2-5사이를 하루에 몇분동안 지속했는지? (유산소 운동 시간)
- speed가 5이상을 하루에 몇분동안 지속했는지?  

In [ ]:
def extract_gps_info(row):
    m_data = row['m_gps']
    altitude = [item['altitude'] for item in m_data]
    latitude = [item['latitude'] for item in m_data]
    longitude = [item['longitude'] for item in m_data]
    speed = [item['speed'] for item in m_data]
    return pd.Series({'altitude': altitude, 'latitude': latitude, 'longitude': longitude, 'speed': speed})

mGps[['altitude','latitude','longitude','speed']] = mGps.apply(extract_gps_info, axis=1)
mGps['lifelog_date'] = mGps['timestamp'].astype(str).str[:10]
mGps = mGps.drop(columns=['m_gps'])
mGps.head(1)

,subject_id,timestamp,altitude,latitude,longitude,speed,lifelog_date
0,id01,2024-06-26 12:03:00,"[110.6, 110.8, 110.8, 110.7, 110.7, 110.8, 110.8, 110.8, 110.8, 110.8, 110.8]","[0.2077385, 0.2077759, 0.2077728, 0.20779, 0.2077914, 0.2077972, 0.2078002, 0.2077985, 0.207801, 0.207802, 0.2078011]","[0.170027, 0.1699851, 0.1699834, 0.1699686, 0.1699708, 0.1699657, 0.1699627, 0.1699631, 0.1699642, 0.1699639, 0.169963]","[0.0, 0.721, 0.0505, 0.6587, 0.0568, 0.1768, 0.0907, 0.0337, 0.0411, 0.0296, 0.0194]",2024-06-26


In [ ]:
# 거리 계산 함수
def haversine(coord1, coord2, unit='m'):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371000  # 지구 반지름(m)
    return c * r if unit == 'm' else c * r / 1000

def process_mGps(df):
    df = df.copy()

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['week'] = df['timestamp'].dt.isocalendar().week

    expanded_rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing GPS data"):
        speeds = ast.literal_eval(row['speed']) if isinstance(row['speed'], str) else row['speed']
        lats = ast.literal_eval(row['latitude']) if isinstance(row['latitude'], str) else row['latitude']
        lons = ast.literal_eval(row['longitude']) if isinstance(row['longitude'], str) else row['longitude']
        alts = ast.literal_eval(row['altitude']) if isinstance(row['altitude'], str) else row['altitude']
        n = len(speeds)
        if n > 0:
            expanded_rows.append(pd.DataFrame({
                'subject_id': [row['subject_id']] * n,
                'lifelog_date': [row['lifelog_date']] * n,
                'timestamp': pd.date_range(start=row['timestamp'], periods=n, freq='1S'),
                'speed': speeds,
                'latitude': lats,
                'longitude': lons,
                'altitude': alts
            }))

    expanded_df = pd.concat(expanded_rows, ignore_index=True)

    # 벡터화
    speeds = expanded_df['speed'].values

    walk_mask = (0.5 <= speeds) & (speeds < 2)
    jog_mask = (2 <= speeds) & (speeds < 5)
    vehicle_mask = (speeds >= 5)
    le5_mask = (speeds <= 5)

    expanded_df['walk'] = walk_mask.astype(int)
    expanded_df['jog'] = jog_mask.astype(int)
    expanded_df['vehicle'] = vehicle_mask.astype(int)
    expanded_df['le5_speed'] = expanded_df['speed'].where(le5_mask)

    # 아침/저녁 구간 조건
    expanded_df['hour'] = expanded_df['timestamp'].dt.hour
    morning_condition = (expanded_df['hour'] >= 6) & (expanded_df['hour'] < 9) & (expanded_df['speed'] >= 1)
    evening_condition = (expanded_df['hour'] >= 21) & (expanded_df['hour'] <= 23) & (expanded_df['speed'] <= 1)

    # 이동 특성 계산
    movement_features = []
    for (subject_id, lifelog_date), group in expanded_df.groupby(['subject_id', 'lifelog_date']):
        all_speeds = group['speed'].values
        all_alts = group['altitude'].values
        all_lats = group['latitude'].values
        all_lons = group['longitude'].values

        active_mins = group.shape[0] / 60  # 1초 단위 → 분
        movement_ratio = (all_speeds > 1.0).mean() if len(all_speeds) > 0 else 0
        alt_change = all_alts[-1] - all_alts[0] if len(all_alts) > 0 else 0
        lat_change = all_lats[-1] - all_lats[0] if len(all_lats) > 0 else 0
        lon_change = all_lons[-1] - all_lons[0] if len(all_lons) > 0 else 0

        total_dist = 0.0
        if len(all_lats) > 1:
            for i in range(len(all_lats)-1):
                coord1 = (all_lats[i], all_lons[i])
                coord2 = (all_lats[i+1], all_lons[i+1])
                total_dist += haversine(coord1, coord2, unit='m')

        movement_features.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date,
            'active_minutes': active_mins,
            'movement_ratio': movement_ratio,
            'alt_change': alt_change,
            'lat_change': lat_change,
            'lon_change': lon_change,
            'total_distance_m': total_dist
        })

    movement_df = pd.DataFrame(movement_features)

    # Groupby + Aggregation
    agg_funcs = {
        'walk_minutes': ('walk', lambda x: x.sum() / 60),
        'jog_minutes': ('jog', lambda x: x.sum() / 60),
        'vehicle_minutes': ('vehicle', lambda x: x.sum() / 60),
        'speed_le5_max': ('le5_speed', 'max'),
        'speed_le5_mean': ('le5_speed', 'mean'),
        'speed_le5_std': ('le5_speed', 'std')
    }

    grouped = expanded_df.groupby(['subject_id', 'lifelog_date']).agg(**agg_funcs).reset_index()
    grouped['exercise_flag'] = np.where(grouped['jog_minutes'] >= 5,1,0)

    # 아침 wakeup time
    morning_first_movement = (
        expanded_df[morning_condition]
        .groupby(['subject_id', 'lifelog_date'])['timestamp']
        .min()
        .reset_index()
        .rename(columns={'timestamp': 'morning_wakeup_time'})
    )


    # 최종 merge
    final = pd.merge(grouped, movement_df, on=['subject_id', 'lifelog_date'], how='left')
    final = pd.merge(final, morning_first_movement, on=['subject_id', 'lifelog_date'], how='left')

    # 아침 wakeup_time 처리
    valid_wakeup = final['morning_wakeup_time'].dropna()
    if not valid_wakeup.empty:
        total_seconds = valid_wakeup.dt.hour * 3600 + valid_wakeup.dt.minute * 60 + valid_wakeup.dt.second
        mean_seconds = total_seconds.mean()
        mean_hour = int(mean_seconds // 3600)
        mean_minute = int((mean_seconds % 3600) // 60)
        mean_second = int(mean_seconds % 60)
        mean_wakeup_time = time(mean_hour, mean_minute, mean_second)
    else:
        mean_wakeup_time = time(7, 0, 0)

    final['morning_wakeup_time'] = final['morning_wakeup_time'].fillna(
        pd.Timestamp.combine(pd.to_datetime('today').date(), mean_wakeup_time)
    )
    final['morning_wakeup_time'] = final['morning_wakeup_time'].dt.hour * 100 + final['morning_wakeup_time'].dt.minute

    mean_wakeup_hhmm = mean_wakeup_time.hour * 100 + mean_wakeup_time.minute

    # wake_up_early_minutes
    def compute_minutes_diff(actual_hhmm, mean_hhmm):
        actual_hour = actual_hhmm // 100
        actual_minute = actual_hhmm % 100
        mean_hour = mean_hhmm // 100
        mean_minute = mean_hhmm % 100
        actual_sec = actual_hour * 3600 + actual_minute * 60
        mean_sec = mean_hour * 3600 + mean_minute * 60
        return (mean_sec - actual_sec) / 60

    final['wake_up_early_minutes'] = final['morning_wakeup_time'].apply(lambda x: compute_minutes_diff(x, mean_wakeup_hhmm))

    return final

In [ ]:
%%time

mGps2 = process_mGps(mGps)

# check
print(f'\n # mGps2 shape: {mGps2.shape}')
mGps2.head(1)

Processing GPS data: 100%|██████████| 800611/800611 [08:24<00:00, 1586.55it/s]



 # mGps2 shape: (660, 17)
CPU times: user 9min 35s, sys: 26.7 s, total: 10min 1s
Wall time: 9min 45s


,subject_id,lifelog_date,walk_minutes,jog_minutes,vehicle_minutes,speed_le5_max,speed_le5_mean,speed_le5_std,exercise_flag,active_minutes,movement_ratio,alt_change,lat_change,lon_change,total_distance_m,morning_wakeup_time,wake_up_early_minutes
0,id01,2024-06-26,11.1667,1.3000,3.8667,4.9907,0.2503,0.5089,0,100.2833,0.1034,-6.7000,0.0229,-0.0757,29113.5760,655,0.0000


### ✔️ mLight 주변 밝기
- Ambient light measured by the smartphone.
 - 어두운 밤	0.1 ~ 1 lux	캄캄한 방, 달빛 없는 밤
 - 가로등 켜진 거리	10 ~ 20 lux	흐릿한 외부 조명
 - 실내 조명	100 ~ 500 lux	사무실, 일반 거실
 - 밝은 실외	10,000 ~ 25,000 lux	맑은 날 햇빛
 - 직사광선 아래	30,000 ~ 100,000 lux	여름 한낮, 매우 강한 햇빛

- 밝기에 따라서 언제 불을 끄고 잠든 시간 추정
- 직사광선 잠에 좋은 영향을 주는지? (논문)
- 결측치 처리 x

In [ ]:
mLight['lifelog_date'] = mLight['timestamp'].astype(str).str[:10]
mLight.head(1)

,subject_id,timestamp,m_light,lifelog_date
0,id01,2024-06-26 12:03:00,534.0000,2024-06-26


In [ ]:
def process_mLight(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6)

    # 하루 요약 통계
    daily_light = df.groupby(['subject_id', 'lifelog_date']).agg(
        light_mean=('m_light', 'mean'),
        light_std=('m_light', 'std'),
        light_max=('m_light', 'max'),
        light_min=('m_light', 'min'),
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()),
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()),
        light_night_ratio=('is_night', 'mean')
    ).reset_index()

    results = []

    for subject_id, group in tqdm(df.groupby('subject_id'), desc="Processing light-based sleep detection"):
        group = group.sort_values('timestamp').reset_index(drop=True)

        recorded_dates = set()
        sleeping = False
        zero_count = 0
        first_zero_time = None

        for i in range(len(group)):
            light = group.loc[i, 'm_light']
            hour = group.loc[i, 'hour']

            if light == 0:
                zero_count += 1
                if zero_count == 1:
                    first_zero_time = group.loc[i, 'timestamp']
                if zero_count >= 6 and not sleeping:
                    sleep_hour = first_zero_time.hour
                    if (sleep_hour >= 21 or sleep_hour <= 2):
                        sleeping = True
            else:
                if sleeping:
                    candidate_wakeup = group.loc[i, 'timestamp']
                    wake_hour = candidate_wakeup.hour

                    if 5 <= wake_hour <= 9 and first_zero_time is not None:
                        wake_time = candidate_wakeup
                        sleep_time = first_zero_time
                        duration_min = (wake_time - sleep_time).total_seconds() / 60

                        if 0 < duration_min <= 840:
                            sleep_duration = duration_min
                        else:
                            sleep_duration = np.nan

                        lifelog_date = wake_time.date() + pd.Timedelta(days=-1)

                        if lifelog_date not in recorded_dates:
                            results.append({
                                'subject_id': subject_id,
                                'lifelog_date': lifelog_date,
                                'sleep_duration_min_mLight': sleep_duration,
                                'sleep_time_min_mLight': sleep_time.hour * 60 + sleep_time.minute,
                                'wake_time_min_mLight': wake_time.hour * 60 + wake_time.minute,
                                'hour_slept_mLight': sleep_time.hour + sleep_time.minute / 60,
                                'hour_woke_up_mLight': wake_time.hour + wake_time.minute / 60
                            })
                            recorded_dates.add(lifelog_date)

                        sleeping = False
                        zero_count = 0
                        first_zero_time = None

            if light > 0:
                zero_count = 0
                first_zero_time = None

    sleep_df = pd.DataFrame(results)

    # 정렬 + 보간
    sleep_df = sleep_df.sort_values(['subject_id', 'lifelog_date'])
    sleep_df['sleep_duration_interp_mLight'] = sleep_df.groupby('subject_id')['sleep_duration_min_mLight'].transform(lambda x: x.interpolate())

    # 시간 단위 파생 컬럼
    sleep_df['sleep_duration_hour_mLight'] = sleep_df['sleep_duration_min_mLight'] / 60
    sleep_df['sleep_duration_interp_hour_mLight'] = sleep_df['sleep_duration_interp_mLight'] / 60

    # 병합
    final = pd.merge(daily_light, sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    return final

In [ ]:
mLight2 = process_mLight(mLight)

# check
print(f'\n # mLight2 shape: {mLight2.shape}')
mLight2.head(1)

Processing light-based sleep detection: 100%|██████████| 10/10 [00:01<00:00,  9.59it/s]



 # mLight2 shape: (700, 17)


,subject_id,lifelog_date,light_mean,light_std,light_max,light_min,light_night_mean,light_day_mean,light_night_ratio,sleep_duration_min_mLight,sleep_time_min_mLight,wake_time_min_mLight,hour_slept_mLight,hour_woke_up_mLight,sleep_duration_interp_mLight,sleep_duration_hour_mLight,sleep_duration_interp_hour_mLight
0,id01,2024-06-26,364.5068,395.6594,1886.0000,0.0000,184.9231,403.4167,0.1781,340.0000,1409.0000,309.0000,23.4833,5.1500,340.0000,5.6667,5.6667


### ✔️ mScreenStatus 화면 사용여부

- Indicates whether the smartphone screen is in use.
 - 기상시간, 취침시간, 수면시간
 - 휴대폰 이용횟수, 이용시간
 - 00 - 05 사이에 휴대폰 이용한 건수
 - 결측치 처리 x

In [ ]:
mScreenStatus['lifelog_date'] = mScreenStatus['timestamp'].astype(str).str[:10]
mScreenStatus.head(1)

,subject_id,timestamp,m_screen_use,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def process_mScreenUse(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour

    # --------------------------------------
    # [1] 하루별 screen 사용 요약
    # --------------------------------------
    features = []

    for (subj, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        status = group['m_screen_use'].values
        ratio_on = status.mean()
        transitions = (status[1:] != status[:-1]).sum()

        durations = []
        current = 0
        for val in status:
            if val == 1:
                current += 1
            elif current > 0:
                durations.append(current)
                current = 0
        if current > 0:
            durations.append(current)

        features.append({
            'subject_id': subj,
            'lifelog_date': lifelog_date,
            'screen_on_ratio': ratio_on,
            'screen_on_transitions': transitions,
            'screen_on_duration_avg': np.mean(durations) if durations else 0,
            'screen_on_duration_max': np.max(durations) if durations else 0,
        })

    daily_screen = pd.DataFrame(features)

    # --------------------------------------
    # [2] 수면시간 추정
    # --------------------------------------
    results = []

    for subject_id, group in tqdm(df.groupby('subject_id'), desc="Processing sleep detection"):
        group = group.sort_values('timestamp').reset_index(drop=True)

        recorded_dates = set()

        sleeping = False
        zero_count = 0
        first_zero_time = None

        for i in range(len(group)):
            screen = group.loc[i, 'm_screen_use']
            hour = group.loc[i, 'hour']

            if screen == 0:
                zero_count += 1
                if zero_count == 1:
                    first_zero_time = group.loc[i, 'timestamp']
                if zero_count >= 120 and not sleeping:
                    sleep_hour = first_zero_time.hour
                    if (sleep_hour >= 21) or (sleep_hour <= 2):
                        sleeping = True
            else:
                if sleeping:
                    candidate_wakeup = group.loc[i, 'timestamp']
                    wake_hour = candidate_wakeup.hour

                    # first_zero_time 존재 확인
                    if 5 <= wake_hour <= 9 and first_zero_time is not None:
                        wake_time = candidate_wakeup
                        sleep_time = first_zero_time
                        duration_min = (wake_time - sleep_time).total_seconds() / 60

                        if 0 < duration_min <= 840: # 14hr
                            sleep_duration = duration_min
                        else:
                            sleep_duration = np.nan

                        lifelog_date = wake_time.date() + pd.Timedelta(days=-1)

                        if lifelog_date not in recorded_dates:
                            results.append({
                                'subject_id': subject_id,
                                'lifelog_date': lifelog_date,
                                'sleep_duration_min_mScreenStatus': sleep_duration,
                                'sleep_time_min_mScreenStatus': sleep_time.hour * 60 + sleep_time.minute,
                                'wake_time_min_mScreenStatus': wake_time.hour * 60 + wake_time.minute,
                                'hour_slept_mScreenStatus': sleep_time.hour + sleep_time.minute / 60,
                                'hour_woke_up_mScreenStatus': wake_time.hour + wake_time.minute / 60
                            })
                            recorded_dates.add(lifelog_date)

                        sleeping = False
                        zero_count = 0
                        first_zero_time = None

            if screen == 1:
                zero_count = 0
                first_zero_time = None

    sleep_df = pd.DataFrame(results)

    # 정렬 + 보간
    sleep_df = sleep_df.sort_values(['subject_id', 'lifelog_date'])
    sleep_df['sleep_duration_interp_mScreenStatus'] = sleep_df.groupby('subject_id')['sleep_duration_min_mScreenStatus'].transform(lambda x: x.interpolate())

    # 시간 단위 수면시간 추가
    sleep_df['sleep_duration_hour_mScreenStatus'] = sleep_df['sleep_duration_min_mScreenStatus'] / 60
    sleep_df['sleep_duration_interp_hour_mScreenStatus'] = sleep_df['sleep_duration_interp_mScreenStatus'] / 60

    # --------------------------------------
    # [3] 최종 merge
    # --------------------------------------
    final = pd.merge(daily_screen, sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    return final

In [ ]:
mScreenStatus2 = process_mScreenUse(mScreenStatus)

# check
print(f'\n # mScreenStatus2 shape: {mScreenStatus2.shape}')
mScreenStatus2.head(1)

Processing sleep detection: 100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


 # mScreenStatus2 shape: (700, 14)


,subject_id,lifelog_date,screen_on_ratio,screen_on_transitions,screen_on_duration_avg,screen_on_duration_max,sleep_duration_min_mScreenStatus,sleep_time_min_mScreenStatus,wake_time_min_mScreenStatus,hour_slept_mScreenStatus,hour_woke_up_mScreenStatus,sleep_duration_interp_mScreenStatus,sleep_duration_hour_mScreenStatus,sleep_duration_interp_hour_mScreenStatus
0,id01,2024-06-26,0.2954,54,7.7778,60,349.0000,1407.0000,316.0000,23.4500,5.2667,349.0000,5.8167,5.8167


### ✔️ mUsageStats 앱사용통계
- mUsageStats: Indicates which apps were used on the smartphone and for how long.

 - 몇시까지 핸드폰 보다가 잠잤는지
 - 통화, 전화 얼마나 했는지
 - YouTube 얼마나 봤는지
 - 메시지, 카카오톡 얼마나 했는지
 - NAVER 얼마나 했는지
 - 평소보다 얼마나 많은 앱을 이용했는지
 - 제외? -> 시스템 UI,One UI 홈

In [ ]:
def extract_mUsageStats_info(row):
    m_data = row['m_usage_stats']
    app_name = [item['app_name'] for item in m_data]
    total_time = [item['total_time'] for item in m_data]
    return pd.Series({'app_name': app_name, 'total_time': total_time})

mUsageStats[['app_name', 'total_time']] = mUsageStats.apply(extract_mUsageStats_info, axis=1)
mUsageStats['lifelog_date'] = mUsageStats['timestamp'].astype(str).str[:10]
mUsageStats.head(1)

,subject_id,timestamp,m_usage_stats,app_name,total_time,lifelog_date
0,id01,2024-06-26 13:00:00,"[{'app_name': ' 캐시워크', 'total_time': 69}, {'app_name': 'NAVER', 'total_time': 549}, {'app_name': ' ✝️성경일독Q', 'total_time': 7337}]","[ 캐시워크, NAVER, ✝️성경일독Q]","[69, 549, 7337]",2024-06-26


In [ ]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]  # '시스템UI'만 제거 (OneUI홈은 포함)

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        last_use = group['timestamp'].max()

        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # daily metrics 생성
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간 구하기
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균대비 화면사용량(%) 생성
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    return daily_stats

In [ ]:
mUsageStats2 = process_mUsageStats(mUsageStats)

# check
print(f'\n # mUsageStats2 shape: {mUsageStats2.shape}')
mUsageStats2.head(1)

### ✔️ mWifi 주변wifi 정보
- Wifi devices around individual subject.
 - -30 ~ -50 dBm	매우 강한 신호 (최적)
 - -51 ~ -60 dBm	강한 신호 (문제 없음)
 - -61 ~ -70 dBm	괜찮은 신호 (약간 느릴 수 있음)
 - -71 ~ -80 dBm	약한 신호 (끊김 주의)
 - -81 dBm 이하	매우 약한 신호 (거의 끊김)

In [ ]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

mWifi[['bssid', 'rssi']] = mWifi.apply(extract_wifi_info, axis=1)
mWifi['lifelog_date'] = mWifi['timestamp'].astype(str).str[:10]
mWifi.head(1)

,subject_id,timestamp,m_wifi,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[{'bssid': 'a0:0f:37:9a:5d:8b', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8c', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8d', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8e', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8f', 'rssi': -78}, {'bssid': 'a0:0f:37:96:56:ef', 'rssi': -58}, {'bssid': '88:36:6c:86:75:84', 'rssi': -72}, {'bssid': 'a0:0f:37:96:56:ee', 'rssi': -58}, {'bssid': 'a0:0f:37:96:56:ed', 'rssi': -58}, {'bssid': '86:25:19:b5:b2:a5', 'rssi': -61}, {'bssid': 'a0:0f:37:96:56:ec', 'rssi': -58}, {'bssid': '1e:39:29:8e:fb:e9', 'rssi': -71}, {'bssid': '52:c2:e8:c7:9b:e4', 'rssi': -82}, {'bssid': 'a0:0f:37:96:56:eb', 'rssi': -58}, {'bssid': '12:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '58:86:94:4a:08:b8', 'rssi': -82}, {'bssid': '90:9f:33:28:d0:2e', 'rssi': -78}, {'bssid': '00:26:66:bc:4e:18', 'rssi': -85}, {'bssid': 'f6:0a:f4:43:4b:ba', 'rssi': -45}, {'bssid': '10:e3:c7:09:20:35', 'rssi': -63}, {'bssid': '10:e3:c7:09:20:34', 'rssi': -89}, {'bssid': '1c:39:29:48:04:92', 'rssi': -82}, {'bssid': '12:e3:c7:07:9d:df', 'rssi': -83}, {'bssid': '86:25:19:c3:44:07', 'rssi': -84}, {'bssid': 'a0:0f:37:9a:37:2f', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2e', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2d', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:ec', 'rssi': -72}, {'bssid': 'a0:0f:37:9a:37:2c', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2b', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:eb', 'rssi': -59}, {'bssid': 'c0:25:2f:d8:c1:a6', 'rssi': -82}, {'bssid': '16:7f:67:bb:fa:f8', 'rssi': -79}, {'bssid': '3c:f3:92:ff:00:01', 'rssi': -82}, {'bssid': '06:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '06:09:b4:74:05:eb', 'rssi': -59}, {'bssid': '12:e3:c7:0a:74:d1', 'rssi': -78}, {'bssid': '88:36:6c:a9:6f:8e', 'rssi': -63}, {'bssid': '02:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '00:09:b4:74:05:eb', 'rssi': -60}, {'bssid': '00:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '00:1d:93:93:cf:fe', 'rssi': -19}, {'bssid': '8e:e2:ac:a5:9d:15', 'rssi': -72}]","[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [ ]:
def process_mWifi(df,threshold):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    def filter_strong_rssi(df,threshold):
        filtered_df = df.copy()
        def filter_row(row):
            bssids = row['bssid']
            rssis = row['rssi']
            # RSSI > threshold 조건 만족하는 항목만 추출
            filtered = [(b, r) for b, r in zip(bssids, rssis) if r > threshold]
            if filtered:
                new_bssids, new_rssis = zip(*filtered)
                return pd.Series({'bssid': list(new_bssids), 'rssi': list(new_rssis)})
            else:
                return pd.Series({'bssid': [], 'rssi': []})
        filtered_df[['bssid', 'rssi']] = filtered_df.apply(filter_row, axis=1)
        return filtered_df

    # === wifi 약신호 제거 ===
    df = filter_strong_rssi(df, threshold=threshold) ####

    features = []
    grouped = df.groupby(['subject_id', 'lifelog_date'])

    for (subject_id, date), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            # 'min_rssi': min_rssi,
            # 'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            'hour_span_minutes': hour_span
        })

    return pd.DataFrame(features)

In [ ]:
mWifi2 = process_mWifi(mWifi,threshold=-60)

# check
print(f'\n # mWifi2 shape: {mWifi2.shape}')
mWifi2.head(1)

### ✔️ wHr 심박동수
- Heart rate readings recorded by the smartwatch.


In [ ]:
wHr['lifelog_date'] = wHr['timestamp'].astype(str).str[:10]
wHr.head(1)

In [ ]:
def get_time_block(hour):
    if 0 <= hour < 6:
        return 'early_morning'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'

def process_wHr_by_timeblock(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['block'] = df['timestamp'].dt.hour.map(get_time_block)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            hr_all = []
            for row in block_group['heart_rate']:
                parsed = ast.literal_eval(row) if isinstance(row, str) else row
                hr_all.extend([int(h) for h in parsed if h is not None])

            if not hr_all:
                continue

            above_100 = [hr for hr in hr_all if hr > 100]
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all)
            block_stats[f'hr_{block}_std'] = np.std(hr_all)
            block_stats[f'hr_{block}_max'] = np.max(hr_all)
            block_stats[f'hr_{block}_min'] = np.min(hr_all)
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wHr2 = process_wHr_by_timeblock(wHr)

# check
print(f'\n # wHr2 shape: {wHr2.shape}')
wHr2.head(1)

### ✔️ wLight 앰비언트 라이트
- Ambient light measured by the smartwatch.  
  - 어두운 밤 0.1 ~ 1 lux 캄캄한 방, 달빛 없는 밤
  - 가로등 켜진 거리 10 ~ 20 lux 흐릿한 외부 조명
  - 실내 조명 100 ~ 500 lux 사무실, 일반 거실
  - 밝은 실외 10,000 ~ 25,000 lux 맑은 날 햇빛
  - 직사광선 아래 30,000 ~ 100,000 lux 여름 한낮, 매우 강한 햇빛

In [ ]:
wLight['lifelog_date'] = wLight['timestamp'].astype(str).str[:10]
wLight.head(1)

In [ ]:
def get_time_block(hour):
    if 0 <= hour < 6:
        return 'early_morning'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'

def process_wLight_by_timeblock(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['block'] = df['timestamp'].dt.hour.map(get_time_block)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'lifelog_date']):
        block_stats = {'subject_id': subj, 'lifelog_date': date}

        for block, block_group in group.groupby('block'):
            lux = block_group['w_light'].dropna().values
            if len(lux) == 0:
                continue

            block_stats[f'wlight_{block}_mean'] = np.mean(lux)
            block_stats[f'wlight_{block}_std'] = np.std(lux)
            block_stats[f'wlight_{block}_max'] = np.max(lux)
            block_stats[f'wlight_{block}_min'] = np.min(lux)

        results.append(block_stats)

    return pd.DataFrame(results)

In [ ]:
wLight2 = process_wLight_by_timeblock(wLight)

# check
print(f'\n # wLight2 shape: {wLight2.shape}')
wLight2.head(1)

### ✔️ wPedo 걸음수
- Step data recorded by the smartwatch.

In [ ]:
wPedo['lifelog_date'] = wPedo['timestamp'].astype(str).str[:10]
wPedo.head(1)

,subject_id,timestamp,step,step_frequency,running_step,walking_step,distance,speed,burned_calories,lifelog_date
0,id01,2024-06-26 12:09:00,10,0.1667,0,0,8.3300,0.1388,0.0000,2024-06-26


In [ ]:
def process_wPedo(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date

    summary = df.groupby(['subject_id', 'lifelog_date']).agg({
        'step': 'sum',
        'step_frequency': 'mean',
        'distance': 'sum',
        'speed': ['mean', 'max'],
        'burned_calories': 'sum'
    }).reset_index()

    # 컬럼 이름 정리
    summary.columns = ['subject_id', 'lifelog_date',
                       'step_sum', 'step_frequency_mean',
                       'distance_sum', 'speed_mean', 'speed_max',
                       'burned_calories_sum']

    return summary

In [ ]:
wPedo2 = process_wPedo(wPedo)

# check
print(f'\n # wPedo2 shape: {wPedo2.shape}')
wPedo2.head(1)


 # wPedo2 shape: (653, 8)


,subject_id,lifelog_date,step_sum,step_frequency_mean,distance_sum,speed_mean,speed_max,burned_calories_sum
0,id01,2024-06-26,3578,0.0927,2782.1901,0.0721,1.5882,189.3191


### 🔥 운동 추정 파생변수

- mActivity 추정행동
- mGps, 핸드폰 위치
- wHr 심박동수
- wPedo 걸음수

In [ ]:
def average_list_columns(df, list_columns, pk_cols=['subject_id', 'lifelog_date']):

    for col in list_columns:

        def safe_mean(x):
            if isinstance(x, list):
                return np.mean(x) if len(x) > 0 else np.nan
            elif isinstance(x, (int, float, np.integer, np.floating, type(None))):
                return x
            elif isinstance(x, (np.ndarray, pd.Series)):
                return np.mean(x)
            elif pd.api.types.is_scalar(x) and pd.isna(x):
                return np.nan
            else:
                return np.nan

        df[col] = df[col].apply(safe_mean)

    return df

def compute_estimated_exercise(mActivity, mGps, wHr, wPedo, minutes):

    # 리스트 평균 처리
    mGps = mGps.copy()
    wHr = wHr.copy()
    mGps = average_list_columns(mGps, ['speed'])
    Hr = average_list_columns(wHr, ['heart_rate'])

    for df in [mActivity, mGps, wHr, wPedo]:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 5분 지속 조건 판단 함수
    def sustained_condition(df, cond_col,minutes):
        df = df[df[cond_col]].sort_values('timestamp')
        times = df['timestamp']
        start = prev = None
        for t in times:
            if start is None:
                start = prev = t
            elif t <= prev + timedelta(minutes=1):
                prev = t
            else:
                if prev - start >= timedelta(minutes=minutes):
                    return True
                start = prev = t
        return (prev - start) >= timedelta(minutes=minutes) if start else False

    # mActivity: m_activity == 7 지속
    mActivity['m_cond'] = mActivity['m_activity'] == 7
    act_flag = mActivity.groupby(['subject_id', 'lifelog_date']) \
                        .apply(lambda df: sustained_condition(df, 'm_cond',40)) \
                        .reset_index(name='act_exe_flag')

    # mGps: speed ∈ [2.5, 5.5] 지속
    mGps['gps_cond'] = mGps['speed'].between(2.5, 5.5)
    gps_flag = mGps.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'gps_cond',minutes)) \
                   .reset_index(name='gps_exe_flag')

    # wHr: hr ≥ 133 상태가 5분 이상 유지
    wHr['whr_cond'] = wHr['heart_rate'] >= 133
    hr_flag = wHr.groupby(['subject_id', 'lifelog_date']) \
                   .apply(lambda df: sustained_condition(df, 'whr_cond',minutes)) \
                   .reset_index(name='hr_exe_flag')

    # wPedo: step ≥ 10000 또는 running_step ≥ 1이 5분 이상
    pedo_daily = wPedo.groupby(['subject_id', 'lifelog_date'])['step'].sum().reset_index(name='total_steps')
    pedo_daily['step_flag'] = pedo_daily['total_steps'] >= 10000

    wPedo['r_cond'] = wPedo['running_step'] >= 1
    run_flag = wPedo.groupby(['subject_id', 'lifelog_date']) \
                    .apply(lambda df: sustained_condition(df, 'r_cond', minutes)) \
                    .reset_index(name='run_flag')

    pedo_flag = pedo_daily.merge(run_flag, on=['subject_id', 'lifelog_date'], how='outer')
    pedo_flag['step_flag'] = pedo_flag['step_flag'].fillna(False)
    pedo_flag['run_flag'] = pedo_flag['run_flag'].fillna(False)
    pedo_flag['pedo_exe_flag'] = pedo_flag[['step_flag', 'run_flag']].any(axis=1)

    # 병합 및 최종 판단
    result = act_flag.merge(gps_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(hr_flag, on=['subject_id', 'lifelog_date'], how='outer') \
                     .merge(pedo_flag[['subject_id', 'lifelog_date', 'pedo_exe_flag']], on=['subject_id', 'lifelog_date'], how='outer')

    # NaN 처리 및 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']:
        result[col] = result[col].fillna(False)

    result['pred_exe_flag'] = result[['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].any(axis=1)

    # 👉 1/0 변환
    for col in ['act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag', 'pred_exe_flag']:
        result[col] = result[col].astype(int)

    display(result[['pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']].sum())

    return result[['subject_id', 'lifelog_date', 'pred_exe_flag', 'act_exe_flag', 'gps_exe_flag', 'hr_exe_flag', 'pedo_exe_flag']]

In [ ]:
# 추정운동여부
exeFlag = compute_estimated_exercise(mActivity, mGps, wHr, wPedo,10)

# check
print(f'\n # exeFlag shape: {exeFlag.shape}')
exeFlag.head(1)

,0
pred_exe_flag,66
act_exe_flag,31
gps_exe_flag,5
hr_exe_flag,18
pedo_exe_flag,19



 # exeFlag shape: (700, 7)


,subject_id,lifelog_date,pred_exe_flag,act_exe_flag,gps_exe_flag,hr_exe_flag,pedo_exe_flag
0,id01,2024-06-26,0,0,0,0,0


### 🔥 Sleeptime 일어난 건수

- Sleeptime에 (mLight 주변 밝기), (wLight 앰비언트 라이트) 변화 건수

In [ ]:
def compute_night_awake_features(df, prefix):

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 00시~06시 필터
    df['hour'] = df['timestamp'].dt.hour
    df_night = df[(df['hour'] >= 0) & (df['hour'] < 6)].copy()

    # 깨어있는 분 계산
    df_night['awake_minute'] = (df_night[prefix] > 0).astype(int)

    # 깨어난 횟수 계산 (0 → 양수 전환)
    def count_awake_blocks(x):
        return ((x > 0) & (x.shift(fill_value=0) == 0)).sum()

    # 그룹별 집계
    result = df_night.groupby(['subject_id', 'lifelog_date']).agg(
        awake_minutes=('awake_minute', 'sum'),
        awake_blocks=(prefix, count_awake_blocks)
    ).reset_index()

    # 컬럼명 변경
    result = result.rename(columns={
        'awake_minutes': f'{prefix}_awake_minutes',
        'awake_blocks': f'{prefix}_awake_blocks'
    })

    # train에 결과 합치기 위해서 -1 day 하기
    result['lifelog_date'] = pd.to_datetime(result['lifelog_date'])
    result['lifelog_date'] = result['lifelog_date'] + pd.Timedelta(days=-1)

    return result

In [ ]:
a1 = compute_night_awake_features(mLight,'m_light')
a2 = compute_night_awake_features(wLight,'w_light')

a1['lifelog_date'] = a1['lifelog_date'].astype(str)
a2['lifelog_date'] = a2['lifelog_date'].astype(str)

sleepWakeCnt = train[['subject_id','lifelog_date']].copy()
sleepWakeCnt = sleepWakeCnt.merge(a1, on=['subject_id','lifelog_date'], how='left')
sleepWakeCnt = sleepWakeCnt.merge(a2, on=['subject_id','lifelog_date'], how='left')

sleepWakeCnt['awake_minutes'] = sleepWakeCnt[['m_light_awake_minutes','w_light_awake_minutes']].max(axis=1)
sleepWakeCnt['awake_blocks'] = sleepWakeCnt[['m_light_awake_blocks','w_light_awake_blocks']].max(axis=1)

# check
sleepWakeCnt.head()

,subject_id,lifelog_date,m_light_awake_minutes,m_light_awake_blocks,w_light_awake_minutes,w_light_awake_blocks,awake_minutes,awake_blocks
0,id01,2024-06-26,4.0000,1.0000,17.0000,3.0000,17.0000,3.0000
1,id01,2024-06-27,4.0000,1.0000,14.0000,3.0000,14.0000,3.0000
2,id01,2024-06-28,4.0000,1.0000,0.0000,0.0000,4.0000,1.0000
3,id01,2024-06-29,1.0000,1.0000,0.0000,0.0000,1.0000,1.0000
4,id01,2024-06-30,2.0000,1.0000,0.0000,0.0000,2.0000,1.0000


### 📦 merge 데이터
- train, test 기간 서로 겹침

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

# 일자변수 타입 변환
mACStatus2['lifelog_date'] = mACStatus2['lifelog_date'].astype(str)
mActivity2['lifelog_date'] = mActivity2['lifelog_date'].astype(str)
mAmbience2['lifelog_date'] = mAmbience2['lifelog_date'].astype(str)
mBle2['lifelog_date'] = mBle2['lifelog_date'].astype(str)
mGps2['lifelog_date'] = mGps2['lifelog_date'].astype(str)
mLight2['lifelog_date'] = mLight2['lifelog_date'].astype(str)
mScreenStatus2['lifelog_date'] = mScreenStatus2['lifelog_date'].astype(str)
mUsageStats2['lifelog_date'] = mUsageStats2['lifelog_date'].astype(str)
mWifi2['lifelog_date'] = mWifi2['lifelog_date'].astype(str)
wHr2['lifelog_date'] = wHr2['lifelog_date'].astype(str)
wLight2['lifelog_date'] = wLight2['lifelog_date'].astype(str)
wPedo2['lifelog_date'] = wPedo2['lifelog_date'].astype(str)

# ---- new ----

exeFlag['lifelog_date'] = exeFlag['lifelog_date'].astype(str)
sleepWakeCnt['lifelog_date'] = sleepWakeCnt['lifelog_date'].astype(str)

In [ ]:
df_list = [
    mACStatus2,       # 1
    mActivity2,       # 2
    mAmbience2,       # 3
    mBle2,            # 4
    mGps2,            # 5
    mLight2,          # 6
    mScreenStatus2,   # 7
    mUsageStats2,     # 8
    mWifi2,           # 9
    wHr2,             # 10
    wLight2,          # 11
    wPedo2,           # 12
    # ---- new ----
    sleepWakeCnt,
    exeFlag
]

data = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'lifelog_date'], how='outer'), df_list)
data['lifelog_date'] = data['lifelog_date'].astype(str)

# 중복체크
print(data.shape)
print(data[['subject_id','lifelog_date']].drop_duplicates().shape)

# merge
train2 = train.merge(data, on=['subject_id','lifelog_date'], how='left')
test2 = test.merge(data, on=['subject_id','lifelog_date'], how='left')

# 저장
print('# train  shape:',train.shape)
print('# train2 shape:',test2.shape)
print('# test   shape:',test.shape)
print('# test2  shape:',test2.shape)
# train2.to_parquet(f"{path}/train2.parquet")
# test2.to_parquet(f"{path}/test2.parquet")

(700, 144)
(700, 2)
# train  shape: (450, 9)
# train2 shape: (250, 151)
# test   shape: (250, 9)
# test2  shape: (250, 151)


### 📌 모델 학습

In [17]:
train = pd.read_parquet(f"{path}/train_0714.parquet")
test = pd.read_parquet(f"{path}/test_0714.parquet")

# drop_features = ['afterwork_max_label','sleeptime_max_label','worktime_max_label']
drop_features = ['top_bssid']
drop_features = [i for i in drop_features if i in train.columns.tolist()]
print('# drop_features:',drop_features)
train = train.drop(columns=drop_features)
test = test.drop(columns=drop_features)

# drop_features: ['top_bssid']


In [18]:
# sleep duration

train['sleep_duration_min_max'] = train[['sleep_duration_min_mScreenStatus','sleep_duration_min_mLight']].max(axis=1)
train['sleep_duration_min_min'] = train[['sleep_duration_min_mScreenStatus','sleep_duration_min_mLight']].min(axis=1)

train['sleep_duration_hour_max'] = train[['sleep_duration_hour_mScreenStatus','sleep_duration_hour_mLight']].max(axis=1)
train['sleep_duration_hour_min'] = train[['sleep_duration_hour_mScreenStatus','sleep_duration_hour_mLight']].min(axis=1)

train['sleep_duration_min_interp_max'] = train[['sleep_duration_interp_mScreenStatus','sleep_duration_interp_mLight']].max(axis=1)
train['sleep_duration_min_interp_min'] = train[['sleep_duration_interp_mScreenStatus','sleep_duration_interp_mLight']].min(axis=1)

train['sleep_time_min_max'] = train[['sleep_time_min_mScreenStatus','sleep_time_min_mLight']].max(axis=1)
train['sleep_time_min_min'] = train[['sleep_time_min_mScreenStatus','sleep_time_min_mLight']].min(axis=1)

test['sleep_duration_min_max'] = test[['sleep_duration_min_mScreenStatus','sleep_duration_min_mLight']].max(axis=1)
test['sleep_duration_min_min'] = test[['sleep_duration_min_mScreenStatus','sleep_duration_min_mLight']].min(axis=1)

test['sleep_duration_hour_max'] = test[['sleep_duration_hour_mScreenStatus','sleep_duration_hour_mLight']].max(axis=1)
test['sleep_duration_hour_min'] = test[['sleep_duration_hour_mScreenStatus','sleep_duration_hour_mLight']].min(axis=1)

test['sleep_duration_min_interp_max'] = test[['sleep_duration_interp_mScreenStatus','sleep_duration_interp_mLight']].max(axis=1)
test['sleep_duration_min_interp_min'] = test[['sleep_duration_interp_mScreenStatus','sleep_duration_interp_mLight']].min(axis=1)

test['sleep_time_min_max'] = test[['sleep_time_min_mScreenStatus','sleep_time_min_mLight']].max(axis=1)
test['sleep_time_min_min'] = test[['sleep_time_min_mScreenStatus','sleep_time_min_mLight']].min(axis=1)

# 요일 컬럼 추가 (예: 월요일, 화요일, ...)
train['lifelog_date'] = pd.to_datetime(train['lifelog_date'])
test['lifelog_date'] = pd.to_datetime(test['lifelog_date'])

# 요일
weekday_map = {
    0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',
    4: '금요일', 5: '토요일', 6: '일요일'
}
train['weekday'] = train['lifelog_date'].dt.dayofweek.map(weekday_map)
test['weekday'] = test['lifelog_date'].dt.dayofweek.map(weekday_map)

# 월
train['month'] = train['lifelog_date'].dt.month
test['month'] = test['lifelog_date'].dt.month

# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','일요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','일요일']),1,0)

# 공휴일
공휴일 = [
     '2024-08-15'
    ,'2024-09-16'
    ,'2024-09-17'
    ,'2024-09-18'
    ,'2024-10-03'
    ,'2024-10-09'
]
train['공휴일'] = np.where(train['lifelog_date'].isin(공휴일),1,0)
test['공휴일'] = np.where(test['lifelog_date'].isin(공휴일),1,0)

# 주말 + 공휴일 묶어주기
# train['weekend'] = np.where( ((train['weekend']==0) & (train['공휴일']==1)), 1, train['weekend'])
# test['weekend'] = np.where( ((test['weekend']==0) & (test['공휴일']==1)), 1, test['weekend'])

In [19]:
def run_basemodel(train, test, common_params, random_state=42, early_stop=False):

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score
    from sklearn.preprocessing import LabelEncoder
    from lightgbm import LGBMClassifier, log_evaluation, early_stopping
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt

    train_df = train.copy()
    test_df = test.copy()

    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    # 노이즈 수준 설정
    def add_noise(series, noise_level, seed=3):
        rng = np.random.default_rng(seed)
        return series * (1 + noise_level * rng.standard_normal(len(series)))

    noise_level = 0.015  # 필요에 따라 조정

    # 타겟인코딩
    # m = 0: 스무딩 없이 범주별 평균만 사용합니다. 관측 수가 많은 범주에는 적합하지만, 적은 경우 과적합 위험이 있습니다.
    # m = 1~10: 일반적인 기본값으로, 대부분의 상황에서 안정적인 성능을 보입니다.
    # m = 50~300: 관측 수가 매우 적은 범주가 많거나 데이터가 희소한 경우에 유용합니다.
    for tgt in all_targets:

      encoder_feats = ['subject_id','month','weekend']

      #### 타겟인코딩1

      subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_te')
      train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
      test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
      global_mean = train_df[tgt].mean()
      test_df[f'{tgt}_te'] = test_df[f'{tgt}_te'].fillna(global_mean)

      # 노이즈 추가
      train_df[f'{tgt}_te'] = add_noise(train_df[f'{tgt}_te'], noise_level)
      test_df[f'{tgt}_te'] = add_noise(test_df[f'{tgt}_te'], noise_level)

      #### 타겟인코딩2

      # 새로운 범주형 열 생성
      train_df['TMP'] = train_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)
      test_df['TMP'] = test_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)

      # 인코더
      encoder = TargetEncoder(cols=['TMP'], smoothing=300) # 40
      encoder.fit(train_df[['TMP']], train_df[tgt])

      # 인코딩 결과를 새로운 열에 저장
      train_df[f'{tgt}_te2'] = encoder.transform(train_df[['TMP']])
      test_df[f'{tgt}_te2'] = encoder.transform(test_df[['TMP']])

      # 노이즈 추가
      train_df[f'{tgt}_te2'] = add_noise(train_df[f'{tgt}_te2'], noise_level)
      test_df[f'{tgt}_te2'] = add_noise(test_df[f'{tgt}_te2'], noise_level)

      # 불필요한 변수 제거
      train_df = train_df.drop(columns=['TMP'])
      test_df = test_df.drop(columns=['TMP'])


    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK]
    print('# categorical_features:',categorical_features)
    for col in categorical_features:
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])


    # X
    X = train_df.drop(columns=PK + all_targets)
    test_X = test_df.drop(columns=PK + all_targets)
    print(f'# X shape: {X.shape}')
    print(f'# test_X shape: {test_X.shape}')

    print('\n STEP1: 실험 결과 확인')
    print("=============== Validation Results ==============")
    total_avg_f1s = []
    best_iteration_temp = {k: [] for k in all_targets}
    f1_by_testsize = {}

    test_size_opt = [0.5, 0.4, 0.3, 0.2]
    for test_size in test_size_opt:
        val_f1 = []
        for col in targets_binary:

            # binary
            y = train_df[col]
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
            best_param = best_param_dict[col].copy()
            best_param['random_state'] = random_state
            model = LGBMClassifier(**best_param)

            if early_stop:
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_train, y_train), (X_valid, y_valid)],
                    callbacks=[early_stopping(stopping_rounds=100,verbose=False)]
                )
                best_iteration_temp[col].append(model.best_iteration_)
            else:
                model.fit(X_train, y_train)
                best_iteration_temp[col].append(1000)

            pred_valid = model.predict(X_valid)
            f1 = f1_score(y_valid, pred_valid)
            val_f1.append(f1)

        # multi
        y = train_df[target_multiclass]
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
        best_param = best_param_dict['S1'].copy()
        best_param['random_state'] = random_state
        model = LGBMClassifier(**best_param, objective='multiclass', num_class=3)

        if early_stop:
            model.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                callbacks=[early_stopping(stopping_rounds=100,verbose=False)]
            )
            best_iteration_temp[target_multiclass].append(model.best_iteration_)
        else:
            model.fit(X_train, y_train)
            best_iteration_temp[target_multiclass].append(1000)

        pred_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, pred_valid, average='macro')
        val_f1.append(f1)

        f1_by_testsize[str(test_size)] = val_f1
        avg_f1 = np.mean(val_f1)
        total_avg_f1s.append(avg_f1)
        detail = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['S1'], val_f1)])
        print(f"[test_size={test_size}] 평균 F1: {avg_f1:.4f} / [상세] {detail}")

    best_iteration_dict = {k: max(best_iteration_temp[k]) for k in all_targets}

    if early_stop==True:
      print("\n[best_iteration_dict]")
      for k, v in best_iteration_dict.items():
          print(f"{k}: {v}")

    print(f"# 전체 평균 F1: {np.mean(total_avg_f1s):.4f}")
    print("================================================")

    # modoling with 100% train & no valid
    print('\n STEP2: 전체 데이터로 모델 재학습')
    print("====== modoling with 100% train & no valid =====")

    # binary
    binary_preds = {}
    for col in targets_binary:
        binary_params = best_param_dict[col].copy()
        binary_params['random_state'] = random_state
        y = train_df[col]

        if early_stop:
          binary_params['n_estimators']=best_iteration_dict[col]
          model = LGBMClassifier(**binary_params)
          model.fit(X, y)
        else:
          model = LGBMClassifier(**binary_params)
          model.fit(X, y)

        binary_preds[col] = model.predict(test_X)
        fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
        top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
        feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
        print(f"[{col}] {feat_str}")

    # multiclass
    y = train_df['S1']
    binary_params = best_param_dict['S1'].copy()
    binary_params['random_state'] = random_state

    if early_stop:
      binary_params['n_estimators']=best_iteration_dict['S1']
      model = LGBMClassifier(**binary_params)
      model.fit(X, y)
    else:
      model = LGBMClassifier(**binary_params)
      model.fit(X, y)

    multiclass_pred = model.predict(test_X)
    fi_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
    feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
    print(f"[S1] {feat_str}")

    # 예측 저장
    submission_final['S1'] = multiclass_pred
    for col in targets_binary:
        submission_final[col] = binary_preds[col]
    submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
    fname = f"submission_{np.mean(total_avg_f1s)}.csv"
    submission_final.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")
    print(f"# submission shape:{submission_final.shape}")
    print("================================================")

    # 모델별 예측결과 비율 비교
    a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    a21 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a23 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a22 = submission_final[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
    result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
    print('\n STEP3: 예측결과 비교표')
    display(result)

In [20]:

"""
# categorical_features: ['weekday']
# X shape: (450, 165)
# test_X shape: (250, 165)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
[test_size=0.5] 평균 F1: 0.7131 / [상세] Q1(기상직후수면질):0.6759 Q2(취침전신체적피로):0.7608 Q3(취침전스트레스):0.7482 S2(수면효율):0.7934 S3(수면잠들기시간):0.7823 S1(S1):0.5181
[test_size=0.4] 평균 F1: 0.7005 / [상세] Q1(기상직후수면질):0.6092 Q2(취침전신체적피로):0.7122 Q3(취침전스트레스):0.7685 S2(수면효율):0.7935 S3(수면잠들기시간):0.7851 S1(S1):0.5345
[test_size=0.3] 평균 F1: 0.7194 / [상세] Q1(기상직후수면질):0.6923 Q2(취침전신체적피로):0.7172 Q3(취침전스트레스):0.7407 S2(수면효율):0.7640 S3(수면잠들기시간):0.8280 S1(S1):0.5739
[test_size=0.2] 평균 F1: 0.7235 / [상세] Q1(기상직후수면질):0.6667 Q2(취침전신체적피로):0.7308 Q3(취침전스트레스):0.7477 S2(수면효율):0.7478 S3(수면잠들기시간):0.8504 S1(S1):0.5976
# 전체 평균 F1: 0.7141
================================================

 STEP2: 전체 데이터로 모델 재학습
====== modoling with 100% train & no valid =====
[Q1] Q1_te2(23), Q1_te(13), wake_time_min_mScreenStatus(12), 통화_time(12), vehicle_minutes(12), hr_morning_above_100_ratio(12), sleep_duration_min_mScreenStatus(11), light_night_mean(8), lat_change(7), ble_rssi_mean_afterwork(6)
[Q2] Q2_te(68), Q2_te2(59), 메신저_time(34), lon_change(33), avg_rssi(33), wake_up_early_minutes(33), screen_time_vs_avg_pct(32), hr_evening_std(30), wlight_evening_mean(30), hr_morning_above_100_ratio(30)
[Q3] Q3_te2(3240), unique_app_count(756), screen_time_vs_avg_pct(735), hr_afternoon_std(717), Q3_te(704), light_max(703), lat_change(690), hr_evening_mean(647), light_night_ratio(645), hr_evening_max(631)
[S2] S2_te(23), S2_te2(17), light_night_mean(16), hr_morning_min(14), wake_time_min_mScreenStatus(10), sleep_duration_interp_hour_mScreenStatus(8), hr_morning_above_100_ratio(5), screen_time_vs_avg_pct(5), ble_class_unknwn_ratio_sleeptime(5), S1_te2(5)
[S3] S3_te(163), S3_te2(25), light_night_mean(20), ble_class_unknwn_ratio_sleeptime(10), max_rssi(9), vehicle_minutes(8), ble_class_others_ratio_sleeptime(7), hr_evening_min(7), ble_rssi_min_afterwork(6), ble_rssi_mean_afterwork(6)
[S1] sleep_duration_min_mScreenStatus(159), wake_time_min_mScreenStatus(131), S1_te2(124), ble_rssi_mean_afterwork(116), ble_rssi_mean_worktime(114), S1_te(111), sleep_duration_min_max(102), screen_time_vs_avg_pct(95), light_night_mean(93), hr_morning_min(86)
# submission_0.714120513489859.csv 저장 완료
# submission shape:(250, 9)
================================================

 STEP3: 예측결과 비교표
학습sum	학습len	학습mean	테스트sum	테스트len	테스트mean
Q1	223	450	0.4956	134	250	0.5360
Q2	253	450	0.5622	149	250	0.5960
Q3	270	450	0.6000	181	250	0.7240
S1	390	450	0.8667	184	250	0.7360
S2	293	450	0.6511	167	250	0.6680
S3	298	450	0.6622	172	250	0.6880
"""

# 공통 하이퍼파라미터
common_params = {
  'n_estimators': 5000,
  "learning_rate": 0.005,
  "shrinkage_rate": 0.12,
  'min_data_in_leaf':2,
  'bagging_fraction':0.9,
  'feature_fraction':0.6,
  'lambda_l1': 5,
  'lambda_l2': 1,
  'max_depth': -1,
  'n_jobs': -1,
  'verbosity': -1
}

# 모델별 세부 하이퍼파라미터
best_param_dict = {'Q1': {'learning_rate': 0.1473150575266255,
  'shrinkage_rate': 0.08585454450680065,
  'min_data_in_leaf': 13,
  'bagging_fraction': 0.5900885111562433,
  'feature_fraction': 0.7398526832500182,
  'lambda_l1': 0.7309384079752819,
  'lambda_l2': 0.010419978985191203,
  'max_depth': 2},
 'Q2': {'learning_rate': 0.1433742819325529,
  'shrinkage_rate': 0.4777741359643458,
  'min_data_in_leaf': 11,
  'bagging_fraction': 0.8942012129234453,
  'feature_fraction': 0.3442323511952453,
  'lambda_l1': 0.11108296857244106,
  'lambda_l2': 0.5000682520529595,
  'max_depth': 11},
 'Q3': {'learning_rate': 0.005440413154494791,
  'shrinkage_rate': 0.4869550654391126,
  'min_data_in_leaf': 5,
  'bagging_fraction': 0.992720410336095,
  'feature_fraction': 0.10854085794750301,
  'lambda_l1': 8.765258863766789,
  'lambda_l2': 0.010911793484805324,
  'max_depth': -1},
 'S1': {'learning_rate': 0.19808502263166988,
  'shrinkage_rate': 0.3292477285579064,
  'min_data_in_leaf': 9,
  'bagging_fraction': 0.5929013243246726,
  'feature_fraction': 0.8481981135327139,
  'lambda_l1': 0.010377995886618164,
  'lambda_l2': 0.6226891522266145,
  'max_depth': 10},
 'S2': {'learning_rate': 0.27099064035077214,
  'shrinkage_rate': 0.028901883938906636,
  'min_data_in_leaf': 9,
  'bagging_fraction': 0.8134249396247819,
  'feature_fraction': 0.2321570003912355,
  'lambda_l1': 8.780092357464005,
  'lambda_l2': 9.605716023562762,
  'max_depth': 7},
 'S3': {'learning_rate': 0.14542046442644,
  'shrinkage_rate': 0.3047247759570036,
  'min_data_in_leaf': 10,
  'bagging_fraction': 0.8493532899163512,
  'feature_fraction': 0.7940889257506005,
  'lambda_l1': 9.299803284110112,
  'lambda_l2': 0.12938944891518922,
  'max_depth': 6}
}

# 공통 하이퍼파라미터 대체 (이상한 모델의 경우)
best_param_dict['Q3'] = common_params

# 전체 평균 F1: 0.7114
# 전체 평균 F1: 0.7145
# 전체 평균 F1: 0.7118
# 전체 평균 F1: 0.7138
# 전체 평균 F1: 0.7141 (BEST PARAMS)
best_iteration_dict = run_basemodel(train, test, best_param_dict, random_state=41, early_stop=False)

# categorical_features: ['weekday']
# X shape: (450, 165)
# test_X shape: (250, 165)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
[test_size=0.5] 평균 F1: 0.7131 / [상세] Q1(기상직후수면질):0.6759 Q2(취침전신체적피로):0.7608 Q3(취침전스트레스):0.7482 S2(수면효율):0.7934 S3(수면잠들기시간):0.7823 S1(S1):0.5181
[test_size=0.4] 평균 F1: 0.7005 / [상세] Q1(기상직후수면질):0.6092 Q2(취침전신체적피로):0.7122 Q3(취침전스트레스):0.7685 S2(수면효율):0.7935 S3(수면잠들기시간):0.7851 S1(S1):0.5345
[test_size=0.3] 평균 F1: 0.7194 / [상세] Q1(기상직후수면질):0.6923 Q2(취침전신체적피로):0.7172 Q3(취침전스트레스):0.7407 S2(수면효율):0.7640 S3(수면잠들기시간):0.8280 S1(S1):0.5739
[test_size=0.2] 평균 F1: 0.7235 / [상세] Q1(기상직후수면질):0.6667 Q2(취침전신체적피로):0.7308 Q3(취침전스트레스):0.7477 S2(수면효율):0.7478 S3(수면잠들기시간):0.8504 S1(S1):0.5976
# 전체 평균 F1: 0.7141

 STEP2: 전체 데이터로 모델 재학습
====== modoling with 100% train & no valid =====
[Q1] Q1_te2(23), Q1_te(13), wake_time_min_mScreenStatus(12), 통화_time(12), vehicle_minutes(12), hr_morning_above_100_ratio(12), sleep_duration_min_mScreenStatus(11), light_night_me

,학습sum,학습len,학습mean,테스트sum,테스트len,테스트mean
Q1,223,450,0.4956,134,250,0.5360
Q2,253,450,0.5622,149,250,0.5960
Q3,270,450,0.6000,181,250,0.7240
S1,390,450,0.8667,184,250,0.7360
S2,293,450,0.6511,167,250,0.6680
S3,298,450,0.6622,172,250,0.6880


### 📌 모델 튜닝

In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

def objective(trial, train_df, target_col):
    params = {
        'n_estimators': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
        'shrinkage_rate': trial.suggest_float('shrinkage_rate', 0.0, 0.5),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.01, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.01, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', -1, 12),
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1
    }

    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK]
    for col in categorical_features:
        train_df[col] = encoder.fit_transform(train_df[col])

    drop_cols = ['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target_col]

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

    if target_col == 'S1':
        params['objective'] = 'multiclass'
        params['num_class'] = 3
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_valid)
        return f1_score(y_valid, preds, average='macro')
    else:
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_valid)
        return f1_score(y_valid, preds)  # 기본은 binary


# 전체 타겟에 대해 튜닝 실행
def run_optuna_all_targets(train_df, targets=['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3'], n_trials=30):
    best_params_all = {}
    for target in targets:
        print(f"\n📌 Start tuning for target: {target}")
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, train_df, target_col=target), n_trials=n_trials)
        print(f"Best F1 for {target}: {study.best_value:.4f}")
        print(f"Best Params for {target}: {study.best_params}")
        best_params_all[target] = study.best_params
    return best_params_all

In [ ]:
# # 하이퍼 파라미터 튜닝
# best_param_dict = run_optuna_all_targets(train, n_trials=30)

In [ ]:
# best_param_dict